# <div style="text-align: center"> DAVIDE MANCINO (847700) - FOUNDATIONS OF COMPUTER SCIENCE</div>
## <div style="text-align: center"> Kiva.org Project </div>
### <div style="text-align: right"> Settembre 2020 </div>

**- Python:** 3.6.10<br>
**- Pandas:** 1.1.2<br>

In [5]:
import pandas as pd
import numpy as np
import datetime
import random
import time

Come prima cosa vengono caricati i dati forniti da Kiva.org in memoria

In [6]:
def load_dataset(name):
    df= pd.read_csv(r'{}'.format(name))
    return df

In [7]:
loans_lenders=load_dataset("./loans_lenders.csv")
loans=load_dataset("./loans.csv")
lenders=load_dataset("./lenders.csv")
country_stats=load_dataset("./country_stats.csv")

In [14]:
loans_lenders.dtypes

loan_id     int64
lenders    object
dtype: object

#### 1) Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

Per riuscire a completare questa consegna si è deciso di utilizzare la funzione *Pandas* **".explode()"** , che permette di trasformare ogni elemento di una lista in una riga, replicando i valori dell'indice. 
Quindi, come prima cosa *loan_id* è stato settato come indice, in seguito *lenders* è stato diviso per la stringa **', '** la lista risultante è stato l'input per la funzione **.explode()** . Infine è stato applicato un reset dell'indice.

In [15]:
loans_lenders_norm = loans_lenders.set_index(['loan_id']).apply(lambda x: x.str.split(', ').explode()).reset_index()
loans_lenders_norm

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


#### 2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

Tramite la funzione **.dtypes** , prima di tutto osserviamo il tipo degli attributi *disburse_time* e *planned_expiration_time*. Dopo aver visto che sono di tipo *object* è stata creata la funzione **get_duration()** dove gli attributi *disburse time* e *planned expiration time* vengono trasformati in formato *date* , è stata così generata la colonna **duration** valorizzata con la differenza tra i due attributi.

In [34]:
loans[['disburse_time', 'planned_expiration_time']].dtypes

disburse_time              object
planned_expiration_time    object
dtype: object

In [42]:
def get_duration():
    loans[['disburse_time', 'planned_expiration_time']]=loans[['disburse_time','planned_expiration_time']].apply(pd.to_datetime).apply(lambda x: x.dt.date)
    loans['duration']=(loans['planned_expiration_time']-loans['disburse_time']).dt.days
    print(loans['duration'])

In [43]:
get_duration()

0          54.0
1          96.0
2          37.0
3          35.0
4          58.0
           ... 
1419602    40.0
1419603    39.0
1419604    51.0
1419605    63.0
1419606    61.0
Name: duration, Length: 1419607, dtype: float64


Infine, sono stati conteggiati i valori *missing* presenti all'interno della colonna **duration** 

In [44]:
print(loans['duration'].isnull().value_counts())

False    1044962
True      374645
Name: duration, dtype: int64


#### 3) Find the lenders that have funded at least twice.

Per riuscire ad estrarre tutti i *lenders* che hanno finanziato *almeno due volte* bisogna effettuare prima di tutto un merge tra dataframe **loans** (filtrando solamente i prestiti con *status== 'funded'*) e la tabella precedentemente creata **loans_lenders_norm**. Successivamente vengono restituiti solo i *lenders* che hanno investito almeno due volte. 

In [78]:
loans['status'].unique()

array(['funded', 'expired', 'refunded', 'fundRaising'], dtype=object)

In [79]:
def lenders_funded_geTwo():
    loans_funded = pd.merge(loans['loan_id'][loans['status'] == 'funded'], loans_lenders_norm, on = 'loan_id')
    print(loans_funded['lenders'].value_counts().loc[lambda x: x>=2])

In [80]:
lenders_funded_geTwo()

gooddogg1           167512
trolltech4460       141268
gmct                123164
nms                  96713
themissionbeltco     74067
                     ...  
sheila4402               2
ron3099                  2
steve61804751            2
tom52703970              2
andreas2888              2
Name: lenders, Length: 893981, dtype: int64


#### 4) For each country, compute how many loans have involved that country as borrowers.

Per calcolare il corretto valore richiesto bisogna contare tutte le occorrenze per ogni Nazione presenti all'interno del campo **country_name** del dataframe *loans*.

In [81]:
loans['country_name'].value_counts()

Philippines         285336
Kenya               143699
Peru                 86000
Cambodia             79701
El Salvador          64037
                     ...  
Uruguay                  1
Canada                   1
Botswana                 1
Mauritania               1
Papua New Guinea         1
Name: country_name, Length: 96, dtype: int64

#### 5) For each country, compute the overall amount of money borrowed.

In questo caso per soddisfare la richiesta abbiamo la necessità di effettuare una **group by** sul campo *country_name* del dataframe *loans* ed effettuare una somma dell'importo preso in prestito *loan_amount*.

In [98]:
country_borrowed = loans.groupby('country_name')['loan_amount'].sum()
country_borrowed

country_name
Afghanistan        1967950.0
Albania            4307350.0
Armenia           22950475.0
Azerbaijan        14784625.0
Belize              150175.0
                     ...    
Vietnam           24681100.0
Virgin Islands       10000.0
Yemen              3444000.0
Zambia             1978975.0
Zimbabwe           5851875.0
Name: loan_amount, Length: 96, dtype: float64

#### 6) Like the previous point, but expressed as a percentage of the overall amount lent.

Viene calcolata la percentuale di denaro preso in prestito sul denaro prestato effettuando il calcolo allo stesso modo della somma per Nazione del *funded_amount* che rappresenterà il nostro denominatore. Infine, calcoliamo la percentuale utilizzando il campo calcolato precedentemente.

In [116]:
country_lent=loans.groupby('country_name')['funded_amount'].sum()
round((country_borrowed/country_lent)*100,2)

country_name
Afghanistan       100.17
Albania           107.89
Armenia           113.14
Azerbaijan        104.18
Belize            100.00
                   ...  
Vietnam           105.19
Virgin Islands       inf
Yemen             103.11
Zambia            103.44
Zimbabwe          100.57
Length: 96, dtype: float64

#### 7) Like the three previous points, but split for each year (with respect to disburse_time).

Come prima operazione viene creata una colonna *disburse_year* all'interno del dataframe **loans** contenente solamente l'anno della rispettiva data di *disburse_time*.

In [100]:
loans['disburse_year']=loans['disburse_time'].apply(lambda x:x.year)

Successivamente andiamo ad effettuare un raggruppamento per *country_name* e *disburse_year* andando a sommare il valore di denaro preso in prestito *loan_amount*. Il risultato è una tabella dove sono presenti i dettagli di denaro preso in prestito per anno per tutte le Nazioni presenti all'interno del dataframe **loans**.

In [101]:
loans.groupby(['country_name', 'disburse_year'])['loan_amount'].sum()

country_name  disburse_year
Afghanistan   2007.0            194975.0
              2008.0            365375.0
              2009.0            585125.0
              2010.0            563350.0
              2011.0            245125.0
                                 ...    
Zimbabwe      2013.0            678525.0
              2014.0           1311575.0
              2015.0            723625.0
              2016.0            788600.0
              2017.0           1237600.0
Name: loan_amount, Length: 748, dtype: float64

Infine, viene calcolata la percentuale di denaro preso in prestito (*loan_amount*) su denaro prestato totale (*funded_amount*) sempre lasciando una aggregazione per Nazione e anno di erogazione.

In [18]:
round((loans.groupby(['country_name', 'disburse_year'])['loan_amount'].sum() / loans['funded_amount'].sum()) * 100,2)

country_name  disburse_year
Afghanistan   2007.0           0.02
              2008.0           0.03
              2009.0           0.05
              2010.0           0.05
              2011.0           0.02
                               ... 
Zimbabwe      2013.0           0.06
              2014.0           0.12
              2015.0           0.06
              2016.0           0.07
              2017.0           0.11
Name: loan_amount, Length: 748, dtype: float64

#### 8) For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount. 

La prima operazione che va effettuata è un *merge* tra il dataframe precentemente normalizzato **loans_lenders_normalized** e **loans**, l'obiettivo è quello di riuscire a creare per ogni *lenders* di ogni *loan_id* il suo contributo assumendo che tutti contribuiscano allo stesso modo.

In [108]:
money_lenders= pd.merge(loans_lenders_norm, loans[['loan_id', 'funded_amount', 'num_lenders_total']],on = 'loan_id')
money_lenders['amount']= money_lenders['funded_amount'] / money_lenders['num_lenders_total']
money_lenders.head()

,loan_id,lenders,funded_amount,num_lenders_total,amount
0,483693,muc888,1225.0,44,27.840909
1,483693,sam4326,1225.0,44,27.840909
2,483693,camaran3922,1225.0,44,27.840909
3,483693,lachheb1865,1225.0,44,27.840909
4,483693,rebecca3499,1225.0,44,27.840909


Infine, per calcolare l'ammontare totale dei soldi prestati da ogni lenders non facciamo altro che sommare tutti gli amount calcolati per lenders.

In [110]:
money_lenders.groupby('lenders')['amount'].sum().round(2).reset_index(name='amount_money_lent')

,lenders,amount_money_lent
0,000,1512.10
1,00000,1249.13
2,0002,2201.18
3,00mike00,38.46
4,0101craign0101,2424.09
...,...,...
1383794,zzmcfate,57535.95
1383795,zzpaghetti9994,44.25
1383796,zzrvmf8538,480.82
1383797,zzzsai,236.89


#### 9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

Dopo aver controllato il numero di *county_code* nulli all'interno dei dataframe **loans** e **lenders** , bisogna creare due funzioni che ci permettano di sostituire i *missing values* con dei *country_code* noti presenti nelle righe vicine.
Questa soluzione è state implementata utilizzando la **random.choice** della libreria **numpy**.

In [120]:
print(loans['country_code'].isna().value_counts())
print(lenders['country_code'].isna().value_counts())

False    1419598
True           9
Name: country_code, dtype: int64
True     1458635
False     890539
Name: country_code, dtype: int64


In [121]:
def fill_null_country_codes(series):
    
    replaces=[]
    
    #tengo solamente i valori non nulli;
    not_null_country_codes=pd.unique(series.dropna())
    replaces.append(not_null_country_codes)
    
    #calcolo la probabilità per ogni country_code;
    replace_probs=series.dropna().value_counts()/len(series.dropna())
    replaces.append(replace_probs)
    
    #identifico gli indici con country_code nullo;
    null_country_code_idxs=series.index[series.isnull()]
    replaces.append(null_country_code_idxs)
    
    #calcolo quanti siano gli indici con country_code nullo;
    replaces.append(len(null_country_code_idxs))
    
    return replaces

In [122]:
def replace_null_country_codes(df,series):
    #richiama la funzione precedente;
    country_codes=fill_null_country_codes(series)
    
    #utilizzo la lista di valori generata da np.random.choice e per index le righe con i country code nulli per creare una Series;
    country_code_replaced=pd.Series(np.random.choice(country_codes[0],p=country_codes[1],size=country_codes[3]),index=country_codes[2])
    
    if(df=="lenders"):
        
        lenders['country_code'].fillna(country_code_replaced,inplace=True)
        return lenders
    
    elif(df=="loans"):
        
        loans['country_code'].fillna(country_code_replaced,inplace=True)
        return loans

In [123]:
lenders=replace_null_country_codes("lenders",lenders['country_code'])
loans=replace_null_country_codes("loans",loans['country_code'])

Dopo aver sostituito i *country_code* nulli si verifica che non ce ne siano più.

In [124]:
print(lenders['country_code'].isna().value_counts())
print(loans['country_code'].isna().value_counts())   

False    2349174
Name: country_code, dtype: int64
False    1419607
Name: country_code, dtype: int64


Adesso possiamo calcolare il valore totale per Nazione dei soldi presi in prestito e dei soldi prestati.
Per il dataframe **loans** basta raggruppare per *country_code* e sommare il *loan_amount*.
Per il dataframe **lenders**, invece, bisogna prima andare in *merge* tra *lenders* e *money_lenders*.

In [133]:
country_money_borrowed= loans.groupby('country_code')['loan_amount'].sum().round(2).reset_index(name = 'amount_money_borrowed')

In [134]:
country_money_lent = pd.merge(money_lenders, lenders[['permanent_name', 'country_code']],left_on = 'lenders', right_on = 'permanent_name')
country_money_lent = country_money_lent.groupby('country_code')['amount'].sum().round(2).reset_index(name = 'amount_money_lent')

Per concludere, si crea un dataframe finale con i valori di soldi prestati e presi in prestito per Nazione *country_code* e viene calcolata la differenza all'interno della colonna **difference** tra soldi prestati **amount_money_lent** meno soldi presi in prestito **amount_money_borrowed**.

In [135]:
country_lent_borrowed = pd.merge(country_money_lent, country_money_borrowed, on='country_code',how='outer')
country_lent_borrowed.fillna(0, inplace=True)
country_lent_borrowed['difference']= round(country_lent_borrowed['amount_money_lent']-country_lent_borrowed['amount_money_borrowed'],2)

In [136]:
country_lent_borrowed

,country_code,amount_money_lent,amount_money_borrowed,difference
0,AD,28235.25,0.0,28235.25
1,AE,2348424.83,0.0,2348424.83
2,AF,100589.28,1967950.0,-1867360.72
3,AG,950.76,0.0,950.76
4,AI,381.27,0.0,381.27
...,...,...,...,...
229,YT,3635.82,0.0,3635.82
230,ZA,515986.97,1006525.0,-490538.03
231,ZM,39791.34,1978975.0,-1939183.66
232,ZW,6552.41,5851875.0,-5845322.59


#### 10) Which country has the highest ratio between the difference computed at the previous point and the population?

La prima operazione per risolvere questo punto è effettuare un *merge* tra il dataframe che contiene le informazioni della popolazione delle varie Nazioni **country_stats** e il dataframe costruito precedentemente **country_lent_borrowed** e viene restituito il Paese che ha la *ratio più alta*.

In [137]:
country_diff_population= pd.merge(country_lent_borrowed,country_stats[['country_code','population']],on='country_code')
country_diff_population= country_diff_population.set_index('country_code')
print((country_diff_population['difference']/country_diff_population['population']).idxmax())

NO


#### 11) Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

Anche questa volta, la prima operazione per risolvere questo punto è effettuare un *merge* tra il dataframe che contiene le informazioni della popolazione e della *population_below_poverty_line* delle varie Nazioni **country_stats** e il dataframe costruito precedentemente **country_lent_borrowed** e viene restituito il Paese che ha la *ratio più alta* stavolta però tenendo conto solamente della popolazione che *non sta sotto la soglia di povertà*.

In [140]:
country_diff_povertyline=pd.merge(country_lent_borrowed, country_stats[['country_code', 'population','population_below_poverty_line']],on='country_code')
country_diff_povertyline['population_above_poverty_line']=(country_diff_povertyline['population']*(100-country_diff_povertyline['population_below_poverty_line'])) / 100
country_diff_povertyline=country_diff_povertyline.set_index('country_code')
print((country_diff_povertyline['difference']/country_diff_povertyline['population_above_poverty_line']).idxmax())

CA


#### 12) For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. 

Per Concludere, per risolvere quest'ultimo punto si sono effettuate alcune operazioni di pulizia dei dati presenti all'interno del dataframe **loans**.
Come prima cosa si è calcolato il valore assoluto della colonna *duration*. Successivamente, si sono eliminati i valori nulli delle colonne *planned_expiration_time* e *disburse_time* e infine, per semplicità di computazione si sono divisi tutti i prestiti con durata diversa da zero da quelli con durata zero.

In [143]:
loans['duration']= loans['duration'].abs()
loans_noNull= loans.dropna(subset=['planned_expiration_time', 'disburse_time'])

duration_noZero= loans_noNull[loans_noNull['duration'] != 0]
duration_zero= loans_noNull[loans_noNull['duration'] == 0]

Viene creata la funzione **day_count** per calcolare il numero di giorni tra la data di inizio e la data fine. Per ogni anno il valore corrisponde al numero di giorni tra la data di inizio e di fine. 

In [152]:
def day_count(date1, date2):
    
    if date1 < date2:
        begin = date1 + datetime.timedelta(1)
        end = date2 + datetime.timedelta(1)
    else:
        begin = date2 + datetime.timedelta(1)
        end = date1 + datetime.timedelta(1)
        
    years = range(begin.year,end.year + 1)
    dictionary = dict()
    
    for year in years:
        year_start = datetime.date(year, 1, 1)
        year_end = datetime.date(year + 1, 1, 1)
        dictionary.update({year:(min(end, year_end) - max(begin, year_start)).days})
        
    return pd.Series(dictionary, name = 'Date_Value')

In [153]:
timetable = duration_noZero.apply(lambda x: day_count(x['disburse_time'], x['planned_expiration_time']), axis = 1)
timetable

,2011,2012,2013,2014,2015,2016,2017,2018
0,NaN,NaN,9.0,45.0,NaN,NaN,NaN,NaN
1,NaN,NaN,11.0,85.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,37.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN
4,NaN,NaN,14.0,44.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1419602,NaN,NaN,NaN,NaN,38.0,2.0,NaN,NaN
1419603,NaN,NaN,NaN,NaN,37.0,2.0,NaN,NaN
1419604,NaN,NaN,NaN,NaN,48.0,3.0,NaN,NaN
1419605,NaN,NaN,NaN,NaN,58.0,5.0,NaN,NaN


Si procede inserendo **0** come valore in tutti i precedenti valori **NaN** del dataframe *timetable*. Successivamente viene effettuato un *merge* tra questo dataframe e quello dei prestiti con durata diversa da zero, per poter ottenere informazioni sulla *duration* e sul *loan_amount*. 

In [154]:
timetable.fillna(0, inplace= True)
timetable= timetable.set_index(duration_noZero['loan_id']).reset_index()
timetable= pd.merge(timetable, duration_noZero[['loan_id','loan_amount','duration']],on='loan_id')
timetable

,loan_id,2011,2012,2013,2014,2015,2016,2017,2018,loan_amount,duration
0,657307,0.0,0.0,9.0,45.0,0.0,0.0,0.0,0.0,125.0,54.0
1,657259,0.0,0.0,11.0,85.0,0.0,0.0,0.0,0.0,400.0,96.0
2,658010,0.0,0.0,0.0,37.0,0.0,0.0,0.0,0.0,400.0,37.0
3,659347,0.0,0.0,0.0,35.0,0.0,0.0,0.0,0.0,625.0,35.0
4,656933,0.0,0.0,14.0,44.0,0.0,0.0,0.0,0.0,425.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...
1044213,988180,0.0,0.0,0.0,0.0,38.0,2.0,0.0,0.0,400.0,40.0
1044214,988213,0.0,0.0,0.0,0.0,37.0,2.0,0.0,0.0,300.0,39.0
1044215,989109,0.0,0.0,0.0,0.0,48.0,3.0,0.0,0.0,2425.0,51.0
1044216,989143,0.0,0.0,0.0,0.0,58.0,5.0,0.0,0.0,100.0,63.0


L'operazione successiva è creare un nuovo dataframe in cui viene calcolato in modo proporzionale l'ammontare dei prestiti per i singoli anni.

In [158]:
loan_amount_per_year = timetable[[2011,2012,2013,2014,2015,2016,2017,2018]].div(timetable['duration'],axis=0).mul(timetable['loan_amount'],axis= 0)
loan_amount_per_year

,2011,2012,2013,2014,2015,2016,2017,2018
0,0.0,0.0,20.833333,104.166667,0.000000,0.000000,0.0,0.0
1,0.0,0.0,45.833333,354.166667,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.000000,400.000000,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.000000,625.000000,0.000000,0.000000,0.0,0.0
4,0.0,0.0,102.586207,322.413793,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...
1044213,0.0,0.0,0.000000,0.000000,380.000000,20.000000,0.0,0.0
1044214,0.0,0.0,0.000000,0.000000,284.615385,15.384615,0.0,0.0
1044215,0.0,0.0,0.000000,0.000000,2282.352941,142.647059,0.0,0.0
1044216,0.0,0.0,0.000000,0.000000,92.063492,7.936508,0.0,0.0


Viene calcolata la somma dei prestiti anno per anno sul dataframe appena creato e anche la somma dei prestiti anno per anno sul dataframe dei prestiti con durata uguale a zero. Infine questi due dataset vengono concatenati, viene eseguita la somma tra i *loan_amount* dei due dataframe anno per anno, ed infine viene stampato il dataframe finale.

In [159]:
l_amount_year_noZero= loan_amount_per_year.sum().to_frame().reset_index().rename(columns={'index':'year', 0:'loan_amount'})
l_amount_year_Zero= duration_zero.groupby('disburse_year')['loan_amount'].sum().to_frame().reset_index().rename(columns={'disburse_year':'year'})

In [174]:
loan_amount_per_year_complete= pd.concat([l_amount_year_noZero,l_amount_year_Zero]).groupby('year')['loan_amount'].sum().to_frame().reset_index()
loan_amount_per_year_complete['loan_amount']= loan_amount_per_year_complete['loan_amount'].round(1)

In [175]:
loan_amount_per_year_complete

,year,loan_amount
0,2011.0,629911.8
1,2012.0,113287580.8
2,2013.0,125015552.0
3,2014.0,153214272.1
4,2015.0,159929916.5
5,2016.0,159363790.4
6,2017.0,172536423.5
7,2018.0,5826652.9
